In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
#import pygsheets
import re

In [2]:
import requests
response = requests.get('https://rotogrinders.com/lineups/mlb?site=fanduel').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response, 'lxml')

In [4]:
import io
with io.open('today.html', 'w', encoding="utf-8") as f:
    f.write(str(soup.prettify()))
    
f.close()

In [5]:
today = soup.find(attrs = {'class':'cmp lineup'})
#today

In [6]:
game_containers = today.find_all(attrs = {'class':'blk crd lineup'})
len(game_containers)

15

In [7]:
away_teams=[]
home_teams=[]
for game in game_containers:
    teams = game.find_all(attrs = {'class':'shrt'})
    away_team = teams[0].text
    away_teams.append(away_team)
    home_team = teams[1].text
    home_teams.append(home_team)

In [8]:
def compile_away_games():
    away_games_list = []
    for game in game_containers:
            teams = game.find_all(attrs = {'class':'shrt'})
            away_team = teams[0].text
            home_team = teams[1].text
            away_players = game.find(attrs={'class': 'blk away-team'})
            away_positions = away_players.find_all(attrs = {'class': 'position'})
            P='P'
            positions = [P] + [i.text.strip() for i in away_positions]
            away_names = away_players.find_all(attrs={'class': 'player-popup'})
            names = [i.text.strip() for i in away_names]
            away_salary = away_players.find_all(attrs={'class':'salary'})
            salary = [i.text.strip() for i in away_salary]
            away_list = [names, positions, salary]
            away_frame = pd.DataFrame({'Team': [away_team] * len(positions), 
                       'vs': ['@ '+home_team] * len(positions), 
                       'POS': positions, 
                       'Player': names, 
                       'Salary': salary})
            away_games_list.append(away_frame)
    return away_games_list

In [9]:
def compile_home_games():
    home_games_list = []
    for game in game_containers:
        teams = game.find_all(attrs = {'class':'shrt'})
        away_team = teams[0].text
        home_team = teams[1].text
        home_players = game.find(attrs={'class': 'blk home-team'})
        home_positions = home_players.find_all(attrs = {'class': 'position'})
        P = 'P'
        positions = [P] + [i.text.strip() for i in home_positions]
        home_names = home_players.find_all(attrs={'class': 'player-popup'})
        names = [i.text.strip() for i in home_names]
        home_salary = home_players.find_all(attrs={'class':'salary'})
        salary = [i.text.strip() for i in home_salary]
        home_frame = pd.DataFrame({'Team': ['@ '+home_team] * len(positions), 
                   'vs': [away_team] * len(positions), 
                   'POS': positions, 
                   'Player': names, 
                   'Salary': salary})
        home_games_list.append(home_frame)
    return home_games_list

In [10]:
compile_home_games()

[    Team   vs   POS             Player Salary
 0  @ NYY  BAL     P    Masahiro Tanaka  $9.4K
 1  @ NYY  BAL    OF      Brett Gardner  $2.7K
 2  @ NYY  BAL    OF        Aaron Judge  $4.6K
 3  @ NYY  BAL    OF  Giancarlo Stanton  $4.3K
 4  @ NYY  BAL  C-1B          Luke Voit  $3.5K
 5  @ NYY  BAL    3B     Miguel Andujar  $3.7K
 6  @ NYY  BAL  C-1B       Gary Sanchez  $3.3K
 7  @ NYY  BAL    2B     Gleyber Torres  $3.3K
 8  @ NYY  BAL  C-1B       Gregory Bird  $2.3K
 9  @ NYY  BAL    SS    Troy Tulowitzki  $2.6K,
     Team   vs   POS          Player  Salary
 0  @ WAS  NYM     P    Max Scherzer  $11.7K
 1  @ WAS  NYM    OF      Adam Eaton   $3.2K
 2  @ WAS  NYM    SS     Trea Turner     $4K
 3  @ WAS  NYM    OF       Juan Soto   $3.7K
 4  @ WAS  NYM    3B  Anthony Rendon   $3.9K
 5  @ WAS  NYM  C-1B  Ryan Zimmerman   $3.1K
 6  @ WAS  NYM    2B    Brian Dozier   $2.9K
 7  @ WAS  NYM  C-1B       Yan Gomes   $2.2K
 8  @ WAS  NYM    OF   Victor Robles     $3K
 9  @ WAS  NYM     P    Max Sche

In [11]:
today_list = []
for i in range(len(compile_away_games())):
    today_list.append(compile_away_games()[i])
    today_list.append(compile_home_games()[i])

In [12]:
today_df = pd.concat(today_list, ignore_index=True)
salaries = today_df.Salary.str.extract(r'(\d[\.]?[\d]?)')
today_df['Salary'] = salaries.astype(np.float)
today_df

,Team,vs,POS,Player,Salary
0,BAL,@ NYY,P,Andrew Cashner,6.0
1,BAL,@ NYY,OF,Cedric Mullins,2.5
2,BAL,@ NYY,OF,Dwight Smith Jr.,2.1
3,BAL,@ NYY,2B,Jonathan Villar,2.8
4,BAL,@ NYY,OF,Trey Mancini,2.5
5,BAL,@ NYY,C-1B,Chris Davis,2.3
6,BAL,@ NYY,OF,Joey Rickard,2.2
7,BAL,@ NYY,3B,Renato Nunez,2.4
8,BAL,@ NYY,C-1B,Pedro Severino,2.0
9,BAL,@ NYY,SS,Richard Martin,2.3
